In [1]:
# データ準備
import pandas as pd
df = pd.read_csv('./breast-cancer-wisconsin-data/data.csv',skiprows=1, header=None)
from sklearn.preprocessing import LabelEncoder
X = df.loc[:, 2:].values
y = df.loc[:,1].values
le = LabelEncoder()
y = le.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=1)

#パイプライン準備
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = Pipeline([('scl',StandardScaler()), ('clf', SVC(random_state=1))])
param_range=[0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0]
param_grid=[{'clf__C':param_range,'clf__kernel':['linear']},
            {'clf__C':param_range,'clf__gamma':param_range,'clf__kernel':['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,scoring='accuracy',cv=10, n_jobs=-1)
gs = gs.fit(X_train,y_train)
print('best score:',gs.best_score_)
print('best params', gs.best_params_)
#print('scores')
#for params, mean_score, scores in gs.grid_scores_:
#    print('%0.3f (+/-%0.3f) for %r' % (mean_score,scores.std(),params))
#print(gs.cv_results_)

# 最適なモデルを使って学習→テストデータで検証
clf = gs.best_estimator_
clf.fit(X_train,y_train)
print('Test accuracy:%.3f' % clf.score(X_test,y_test))

best score: 0.978021978022
best params {'clf__C': 0.1, 'clf__kernel': 'linear'}
Test accuracy:0.965


In [5]:
import numpy as np
from sklearn.model_selection import cross_val_score

gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1)
scores = cross_val_score(gs, X_train, y_train, scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' %( np.mean(scores), np.std(scores)))


CV accuracy: 0.965 +/- 0.025


In [7]:
from sklearn.tree import DecisionTreeClassifier

gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                 param_grid=[{'max_depth':[12,3,4,5,6,7,None]}],
                 scoring='accuracy',
                 cv=2)
scores = cross_val_score(gs,
                        X_train,
                        y_train,
                        scoring='accuracy',
                        cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.930 +/- 0.020
